# importing the libraries

In [1]:
import os
import sys
import tensorflow as tf
import skimage.io as io
import numpy as np
import scipy
import cv2
import dlib
#sys.path.append('/Users/yu-chieh/seg_models/models/slim/')
#sys.path.append("/Users/yu-chieh/seg_models/tf-image-segmentation/")
#from tf_image_segmentation.models.fcn_8s import FCN_8s
#from tf_image_segmentation.utils.inference import adapt_network_for_any_size_input
from scipy import misc
import matplotlib.pyplot as plt

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Importing data and Preprocessing

In [2]:
#defining a function that returns certain no of images as numpy arrays for further preprocessing
def get_all_images_for_fcn(num_images, path):
    # get num_images images form the path and put as a matrix
    imgs = []
    num = 0
    for f in os.listdir(path): #iterating on all the files in the directory
        print(f)
        if num >= num_images:
            return np.array(imgs)
        image_path = os.path.join(path,f)
        image = scipy.ndimage.imread(image_path, mode='RGB') #reading images in RGB mode
        # cheating version
        # image = np.dstack((image, get_xy_mask(image)))
        imgs.append(image)  #storing the images as numpy array
        num += 1
    return np.array(imgs)

now our main job is to detect the face or important feature points of the face in the image and apply a homography transform to project the face on another co-ordinate plane in such a way that the transformed image is centered around the face and the label of every pixel is detected calculating the distance between the origin(face) and that pixel. By this channel we can centralize the face in the image that gives us better segmentation result

In [3]:
#Function detecting facial point
#this function takes a image as a input and retruns the co-ordinates of the main facial feature points

def get_facial_points(image, num_points):
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    detector = dlib.get_frontal_face_detector() #creats a face detector object
    dets = detector(image, 1)
    win = dlib.image_window()
    win.clear_overlay()
    win.set_image(image)
    points = []
    for k, d in enumerate(dets):
        # Get the landmarks/parts for the face in box d.
        shape = predictor(image, d)
        for i in range(num_points):
            pt = shape.part(i)
            points.append([int(pt.x), int(pt.y)])
        win.add_overlay(shape)
    win.add_overlay(dets)
    dlib.hit_enter_to_continue()
    return np.array(points)

In [4]:
path ='C:/Users/hp/Desktop/ML Projects/Portrait Segmentation/dataset/training/'

# Creating the Normalized x-y channel

In [5]:
def get_xy_mask(img_path,mask_path):
    image_src = scipy.ndimage.imread(img_path,mode = 'RGB')
    mask_dst = scipy.ndimage.imread(mask_path, mode = 'RGB')
    dst = get_facial_points(mask_dst, 68)
    src = get_facial_points(image_src, 68)
    h, status = cv2.findHomography(src, dst)
    im_dst = cv2.warpPerspective(image_src, h, (image_src.shape[1], image_src.shape[0]))
    return im_dst

# Applying PCA

for reducing the dimensionality keeping most of it's variant intact

In [6]:
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA as RandomizedPCA
def apply_PCA(image):
    pca = RandomizedPCA(200)
    pca.fit(image)
    components = pca.transform(image)
    projected = pca.inverse_transform(components)
    projected = projected.flatten().reshape(3,800,400)
    trans_image = projected
    return trans_image